In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [348]:
#load the filtered data 
df = pd.read_csv("L:\\ML-Assignment\\filtered_data.csv")

In [349]:
#split the data into 2 classes according to the sentiment levels 
splits = list(df.groupby("Sentiment Level"))


negative = splits[0][1]
positive = splits[1][1]

In [350]:
#randomly select 50000 features from each class 
import random as rand

number = rand.randint(0,500000)
positive = positive[number : number + 50000]

number = rand.randint(0,500000)
negative = negative[number: number + 50000]

In [351]:
#concat 2 classes into 1 dataframe
df_concat = pd.concat([positive, negative], axis = 0)

#shuffle the sequence of the dataframe
df_concat = df_concat.sample(frac = 1)

In [353]:
#Split dataframes into random train and test subsets, the method returns lists
#default ratio in splitting the training and testing set is 3:1 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_concat["joined_Tweet"], df_concat["Sentiment Level"], random_state = 0)

In [354]:
#convert the lists into numpy arrays of type string 
x_train = np.asarray(x_train.to_frame().to_numpy(dtype=np.string_)).astype(np.string_)
x_test = np.asarray(x_test.to_frame().to_numpy(dtype=np.string_)).astype(np.string_)
y_train = np.asarray(y_train.to_frame().to_numpy(dtype=np.string_).astype(np.string_))
y_test = np.asarray(y_test.to_frame().to_numpy(dtype=np.string_).astype(np.string_))


#create labels in the form of a 2 dimensional array to split the labels into 2 distinct classes 
def create_label(y):
    counter = 0
    result = np.array([[]])
    if y[0] == np.string_("0"):
        result = np.array([[1,0]])
    elif y[0] == np.string_("4"):
        result = np.array([[0,1]])
    
    for i in y: 
        if counter > 0:
            if i == np.string_("0"):
                result = np.append(result, np.array([[1,0]]), axis = 0)
            elif i == np.string_("4"):
                result = np.append(result, np.array([[0,1]]), axis = 0)
        counter+=1
    

    return result


#create labels from the y_train and y_test 
new_y_train = create_label(y_train)   
new_y_test = create_label(y_test)

                

In [10]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import Dense, TextVectorization
from sklearn.metrics import accuracy_score

In [368]:
#create our machine learning model 
model = Sequential(name = "Nice_model")
model.add(Input(shape=(1,), dtype = tf.string))

#text vectorization is needed to convert the texts into integers 
vectorize_layer = TextVectorization(split = "whitespace", 
                                    ngrams=(1,2),
                                    output_mode = "tf_idf")

vectorize_layer.adapt(np.asarray(df_concat["joined_Tweet"].to_numpy(dtype=np.string_)).astype(np.string_))


#add the layers into our models 
model.add(vectorize_layer)

model.add(Dense(units = 128, activation = "relu", name = "L1"))
model.add(Dense(units = 128, activation = "relu", name = "L2"))
model.add(Dense(units = 128, activation = "relu", name = "L3"))
model.add(Dense(units = 64, activation = "relu", name = "L4"))
model.add(Dense(units = 2, activation = "sigmoid", name = "Output"))

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics = "accuracy")

print("OK")

OK


In [369]:
model.summary()

Model: "Nice_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_17 (Text  (None, 484946)           1         
 Vectorization)                                                  
                                                                 
 L1 (Dense)                  (None, 128)               62073216  
                                                                 
 L2 (Dense)                  (None, 128)               16512     
                                                                 
 L3 (Dense)                  (None, 128)               16512     
                                                                 
 L4 (Dense)                  (None, 64)                8256      
                                                                 
 Output (Dense)              (None, 2)                 130       
                                                        

In [ ]:
#display the score of our model 
score = model.evaluate(x_test, new_y_test)

print(score[0])
print(score[1])

In [373]:
#start the training here 
log_dir = "L:\\ML-Assignment\\Logs\\Logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram freq = 1)

model.fit(x_train, new_y_train,
         epochs=5,
         batch_size=50,
         callbacks=[tensorboard_callback],
         verbose = 1,
         validation_data = (x_test, new_y_test))


Epoch 1/5
1500/1500 [==============================] - 912s 608ms/step - loss: 0.4643 - accuracy: 0.7892 - val_loss: 0.4478 - val_accuracy: 0.8005
Epoch 2/5
1500/1500 [==============================] - 817s 544ms/step - loss: 0.2894 - accuracy: 0.8879 - val_loss: 0.4888 - val_accuracy: 0.7947
Epoch 3/5
1500/1500 [==============================] - 827s 551ms/step - loss: 0.1394 - accuracy: 0.9544 - val_loss: 0.5938 - val_accuracy: 0.7895
Epoch 4/5
1500/1500 [==============================] - 831s 554ms/step - loss: 0.0707 - accuracy: 0.9794 - val_loss: 0.7775 - val_accuracy: 0.7835
Epoch 5/5
1500/1500 [==============================] - 811s 541ms/step - loss: 0.0495 - accuracy: 0.9860 - val_loss: 0.8972 - val_accuracy: 0.7750


In [7]:
#%reload_ext tensorboard
#%tensorboard --logdir logs/fit

In [12]:
#save the model, remember to change the model number in the file name 
model_dir = "L:\\ML-Assignment\\Model\\Model_02"
model.save(model_dir)

In [376]:
#delete the model after it is saved 
del model

In [13]:
#load the model 
model = load_model(model_dir)

print("Loaded")

Loaded


In [21]:
#clean the input like how we cleaned the data for the model 
import contractions
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np 

stopwords = set(stopwords.words('english'))

def predict(model, input, stopword = stopwords):
    input = input.lower()
    input = re.sub("@[^\s]+", "", input)
    input = re.sub('r[^\w\s]+', "", input)
    input = input.split(" ")

    input = [x.encode("ascii","ignore").decode() for x in input]
    input = [contractions.fix(x) for x in input]
    input = ' '.join(map(str, input))
    input = word_tokenize(input, language = 'english')
    #input = [x for x in input if x not in stopwords]
    input = [x for x in input if not x.isdigit()]
    input = ' '.join(map(str, input))
    filtered_text = input
    
    x_input = np.array([input], dtype = np.string_)
    y = model.predict(x_input)
    result = ""
    
    if y[0][0] == y[0].max():
        result = "High probability of depression"
    elif y[0][1] == y[0].max():
        result = "Low probabiltity of depression"
        
    return (result, y, filtered_text)

#return a list wit


In [28]:
#test the model with inputs 
while True: 
    x = input("Enter Text: ")
    
    if x == "quit" or x == "exit":
        break
    
    y = predict(model, x)
    print("Filtered Text: " + y[2])
    print("Model Result: " + y[0])
    print("Probability of depression: ", "{:.4f}".format(y[1][0][0] * 100))

    
print("Completed.")

Enter Text: exit
Completed.
